In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace, LossDict, MAEDict
import gc
import wandb 
from fastai.callback.wandb import WandbCallback
from fastxtend.vision.all import EMACallback
import pickle

In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    pl_data = Path('../data/rna_files/final0_PLfolds_ft_tot.pickle')
    bs = 32
    num_workers = 4
    device = 'cuda'
    seed = 2023
    out = 'exp_32_psd_v2'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV6SAVEDwithFMPSD'
    dataset_val = 'RNA_DatasetBaselineSplitssbppV6SAVEDwithFM'
    sn_train = False

    
    model_name = 'RNA_ModelV25'
    model_kwargs = dict(dim=192 * 2,
        depth=4,
        head_size=32,
        drop_pat_dropout=0.2,
        dropout=0.2,
        bpp_transfomer_depth = 4)

    epoch = 9
    lr = 5e-5
    wd = 0.05
    pct_start = 0.01
    
    
    md_wt = 'exp_32_psd_v1/models/model.pth'
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

In [4]:
def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [5]:



with open(str(CFG.pl_data), "rb") as f: 
    data = pickle.load(f)
orig_test_csv = CFG.path/'test_sequences.csv'
train_data = CFG.path/'train_data.parquet'
folds_split = CFG.split_id



ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(data, orig_test_csv, train_data, folds_split, mode='train')

dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_size=CFG.bs,
            drop_last=True, 
            shuffle =True,
            num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)




split = pd.read_csv(CFG.split_id)
df = pd.read_parquet(CFG.path/'train_corrected.parquet')
df = pd.merge(df, split, on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)

ds_val = getattr(rnacomp.dataset, CFG.dataset_val)(df_valid, mode='eval')
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val,  
                                                     batch_size=CFG.bs, 
                                                       drop_last=False,
                                                     num_workers=CFG.num_workers),
                         CFG.device)


data = DataLoaders(dl_train,dl_val)
del split
del df
gc.collect()



8027

In [6]:
#the training was interepeted on epoch 3 
#so i had to restart from scratch but loading weights from epoch 3
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(**CFG.model_kwargs).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    #WandbCallback(log_preds=False),
                    CSVLogger(),
                     EMACallback(replace_weights=True),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.model.load_state_dict(torch.load(CFG.md_wt), strict=False)
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)




epoch,train_loss,valid_loss,mae,time
0,0.024888,0.113108,0.123446,3:14:56
1,0.025237,0.113053,0.123382,3:17:47
2,0.024142,0.112943,0.123271,3:18:46
3,0.023683,0.112819,0.123133,3:18:51
4,0.023559,0.112704,0.123003,3:19:21
5,0.023436,0.112633,0.122937,3:19:08
6,0.022291,0.112566,0.122861,3:19:08


Better model found at epoch 0 with mae value: 0.12344614007709559.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Better model found at epoch 1 with mae value: 0.12338182507436941.
Better model found at epoch 2 with mae value: 0.12327148397408094.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Better model found at epoch 5 with mae value: 0.12293705937201593.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Better model found at epoch 6 with mae value: 0.12286141583015334.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



BadZipFile: Caught BadZipFile in DataLoader worker process 3.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/slh/rna/exp/../rnacomp/dataset.py", line 3433, in __getitem__
    bpp_extra = [
  File "/opt/slh/rna/exp/../rnacomp/dataset.py", line 3434, in <listcomp>
    torch.from_numpy(data[i].astype(np.float32)) for i in self.extra_bpp
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 256, in __getitem__
    return format.read_array(bytes,
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
from tqdm import tqdm 
for x, y in tqdm(dl_val):
    x

 66%|███████████████████████████████████████████████████████████▋                               | 736/1123 [00:29<00:13, 29.05it/s]

In [7]:
k

NameError: name 'k' is not defined

### 